<a href="https://colab.research.google.com/github/brendanlooker/colab-examples/blob/main/looker/Rotate_BQ_SA_Key_in_Looker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install \
  google-api-python-client \
  google-auth \
  google-auth-httplib2 \
  google-cloud-secret-manager \
  looker_sdk


In [ ]:
import base64
import json
import os
from googleapiclient import discovery
from google.cloud import secretmanager
import google.auth
import looker_sdk
from looker_sdk import models40

In [ ]:
# Authenticate using ADC

from google.colab import auth
auth.authenticate_user()


In [ ]:
# Configure some variables

PROJECT_ID = "brendanlooker"
SERVICE_ACCOUNT_NAME = "looker"
SERVICE_ACCOUNT_ID = f"{SERVICE_ACCOUNT_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"
SECRET_ID = "looker-bq-sa-key"
LOOKER_BQ_CONNECTION_NAME = "thelook"


In [ ]:
# Create a Service Account Key - Assumes that the Servive Account already exists

def create_service_account_key():
    service = discovery.build('iam', 'v1')
    name = f'projects/{PROJECT_ID}/serviceAccounts/{SERVICE_ACCOUNT_ID}'

    key = service.projects().serviceAccounts().keys().create(
        name=name,
        body={"privateKeyType": "TYPE_GOOGLE_CREDENTIALS_FILE", "keyAlgorithm": "KEY_ALG_RSA_2048"}
    ).execute()

    key_data = key['privateKeyData']
    decoded_key = json.loads(base64.b64decode(key_data).decode("utf-8"))

    return decoded_key

key = create_service_account_key()


In [ ]:
# Store the Key in Secret Manager - Assumes the Secret ID already exists

def store_secret(key):
    client = secretmanager.SecretManagerServiceClient()
    parent = f"projects/{PROJECT_ID}"

    try:
        client.create_secret(
            request={
                "parent": parent,
                "secret_id": SECRET_ID,
                "secret": {"replication": {"automatic": {}}}
            }
        )
    except Exception:
        pass

    payload = json.dumps(key).encode("utf-8")
    client.add_secret_version(
        request={
            "parent": f"{parent}/secrets/{SECRET_ID}",
            "payload": {"data": payload}
        }
    )
    print("Stored service account key in Secret Manager.")

store_secret(key)

In [ ]:
# Config Looker Connection

from getpass import getpass

# Looker instance details
os.environ["LOOKERSDK_BASE_URL"] = "https://0a008a8e-ad98-4f2e-95c4-99bcdc1ff974.looker.app"       # Update to reference valis Looker instance
os.environ["LOOKERSDK_CLIENT_ID"] = input("Enter Looker Client ID: ")          # Add API Client ID
os.environ["LOOKERSDK_CLIENT_SECRET"] = getpass("Enter Looker Client Secret: ") # Add API Client ID



In [ ]:
# Test Authenticatin to Looker

sdk = looker_sdk.init40()

me = sdk.me(fields="email")
print(me.email)

In [ ]:
# Get Key from Secret Manager
# Looker requires the Creds to be base64 encoded

def get_encoded_creds_from_secret(project_id: str, secret_id: str) -> str:
    client = secretmanager.SecretManagerServiceClient()
    secret_path = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(name=secret_path)

    json_str = response.payload.data.decode("utf-8")  # JSON string of SA key
    encoded_creds = base64.b64encode(json_str.encode("utf-8")).decode("utf-8")
    return encoded_creds

new_key = get_encoded_creds_from_secret('brendanlooker', 'looker-bq-sa-key')

In [ ]:

# Fetch existing connection
existing_conn = sdk.connection(connection_name=LOOKER_BQ_CONNECTION_NAME)

conn_dict = {
    k: v for k, v in existing_conn.__dict__.items() if not k.startswith("_")
}


# Fields you CANNOT set on creation (read-only)
read_only_fields = [
    "id", "created_at", "user_id", "dialect", "last_metadata_sync",
    "last_reclaim_errors", "last_regen_error", "last_reclaim_date"
]
for field in read_only_fields:
    conn_dict.pop(field, None)

# Inject new creds and specify the file type
conn_dict["certificate"] = new_key  # Inject new creds
conn_dict["file_type"] = "json" # Set the file type to json


# Delete the existing connection
try:
    sdk.delete_connection(connection_name=LOOKER_BQ_CONNECTION_NAME)
    print(f"Deleted existing connection: {LOOKER_BQ_CONNECTION_NAME}")
except Exception as e:
    print(f"Warning: {e}")

# Create new Looker Connection using the updated dictionary
new_conn = models40.DBConnection(**conn_dict)
sdk.create_connection(new_conn)
print(f"Recreated connection '{LOOKER_BQ_CONNECTION_NAME}' with rotated credentials.")
